From [here](https://www2.karlin.mff.cuni.cz/~otava/ClinExpSolution/assignment3_solution.html). The matching algorithm in R is [here](https://sejdemyr.github.io/r-tutorials/statistics/tutorial8.html).

The Los Angeles Study of Endometrial Cancer was a matched case-control study conducted in California in the 1970’s. There are 63 cases of endometrial cancer, all women age 55 or over, each matched to four controls living in the same retirement community. The primary exposure of interest was estrogen use. The secondary exposure was gallbladder disease. The Epi library in R includes two versions of the data: the full dataset `bdendo` and a subset containing a single control matched to each case `bdendo11`.

```
set Case-control set: a numeric vector
d   Case or control: a numeric vector (1=case, 0=control)
gall  Gall bladder disease
hyp 	Hypertension
ob    Obesity
est   Estrogen therapy
dur   Duration of conjugated estrogen therapy: a factor with levels 0, 1, 2, 3, 4.
non   Use of non estrogen drugs: a factor with levels No Yes.
duration Months of oestrogen therapy: a numeric vector.
cest  Conjugated oestrogen dose a factor with levels 0, 1, 2, 3
```


In [ ]:
install.packages('Epi')
library('Epi')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘RcppArmadillo’, ‘cmprsk’, ‘etm’, ‘plyr’, ‘numDeriv’, ‘zoo’




In [ ]:
data(bdendo11)
head(bdendo11)

,set,d,gall,hyp,ob,est,dur,non,duration,age,cest,agegrp,age3
,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>
1,1,1,No,No,Yes,Yes,4,Yes,96,74,3,70-74,65-74
2,1,0,No,No,NA,No,0,No,0,75,0,70-74,65-74
3,1,0,No,No,NA,No,0,No,0,74,0,70-74,65-74
4,1,0,No,No,NA,No,0,No,0,74,0,70-74,65-74
5,1,0,No,No,Yes,Yes,3,Yes,48,75,1,70-74,65-74
6,2,1,No,No,No,Yes,4,Yes,96,67,3,65-69,65-74


In [ ]:
data(bdendo11)

table(as.character(bdendo11$d))
with(bdendo11, twoby2(d,est))


 0  1 
63 63 

2 by 2 table analysis: 
------------------------------------------------------ 
Outcome   : No 
Comparing : 0 vs. 1 

  No Yes    P(No) 95% conf. interval
0 33  30   0.5238    0.4015   0.6433
1  7  56   0.1111    0.0539   0.2152

                                   95% conf. interval
             Relative Risk: 4.7143    2.2559   9.8517
         Sample Odds Ratio: 8.8000    3.4778  22.2669
Conditional MLE Odds Ratio: 8.6344    3.2632  26.0178
    Probability difference: 0.4127    0.2550   0.5437

             Exact P-value: 0.0000 
        Asymptotic P-value: 0.0000 
------------------------------------------------------


In [ ]:
with(bdendo11, twoby2(d,gall))

2 by 2 table analysis: 
------------------------------------------------------ 
Outcome   : No 
Comparing : 0 vs. 1 

  No Yes    P(No) 95% conf. interval
0 54   9   0.8571    0.7476   0.9240
1 46  17   0.7302    0.6080   0.8252

                                   95% conf. interval
             Relative Risk: 1.1739    0.9797   1.4066
         Sample Odds Ratio: 2.2174    0.9028   5.4462
Conditional MLE Odds Ratio: 2.2035    0.8346   6.1824
    Probability difference: 0.1270   -0.0154   0.2643

             Exact P-value: 0.1223 
        Asymptotic P-value: 0.0824 
------------------------------------------------------


Now, let us calculate the odds ratios using mh function (Mantel-Haenszel analyses of cohort and case-control studies) for 1:1 matching. Unlike in analysis above, here we consider the matching and therefore the resulting odds ratio is different.

In [ ]:
tg11Est = with(bdendo11,table(d, est, set))
mhor11Est = mh(tg11Est["1",,], tg11Est["0",,], compare = "est", levels = c(2, 1))
outEst <- c(mhor11Est$ratio, mhor11Est$cl.lower, mhor11Est$cl.upper, mhor11Est$p.value)
names(outEst) <- c("Odds ratio", "CI lower", "CI upper", "p-val")
round(outEst,3)

Odds ratio   CI lower   CI upper      p-val 
     9.667      3.565     26.213      0.000

In [ ]:
fit <- clogistic(d ~ gall + hyp + cest + dur, strata = set, data =bdendo11)

In [ ]:
fit


Call: 
clogistic(formula = d ~ gall + hyp + cest + dur, strata = set, 
    data = bdendo11)




          coef exp(coef) se(coef)      z     p
gallYes  0.611     1.842    0.701  0.871 0.380
hypYes   0.198     1.218    0.487  0.406 0.680
cest1    1.481     4.399    0.835  1.775 0.076
cest2    0.677     1.968    0.853  0.793 0.430
cest3    1.960     7.096    0.908  2.158 0.031
dur1    -0.483     0.617    0.779 -0.620 0.540
dur2     1.090     2.975    0.926  1.177 0.240
dur3     0.122     1.130    0.839  0.146 0.880
dur4        NA        NA    0.000     NA    NA

Likelihood ratio test=21.2  on 8 df, p=0.0066, n=100